In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Load the dataset
movies_df = pd.read_csv('movies.csv')

user_movie_name = input("Enter a movie name: ")

while user_movie_name.strip() == '' or user_movie_name not in movies_df['title'].values:
    if user_movie_name.strip() == '':
        try:
            user_director_name = input("Enter the director's name: ")
            if user_movie_name.strip() == '' and user_director_name.strip() == '':
                print("You didn't pick anything.")
                break
            director_movies = movies_df[movies_df['director'] == user_director_name]['title'].tolist()
            if director_movies:
                print(f"Movies directed by {user_director_name}:")
                for movie_title in director_movies:
                    movie_row = movies_df[movies_df['title'] == movie_title]
                    homepage = movie_row['homepage'].values[0] if not movie_row['homepage'].empty else "No homepage available"
                    cast = movie_row['cast'].values[0] if not movie_row['cast'].empty else "No cast information available"
                    print(f"{movie_title} (Homepage: {homepage}, Cast: {cast})")
                exit()
            else:
                print(f"No movies found for director {user_director_name}.")
        except KeyboardInterrupt:
            print("\nYou didn't pick anything.")
            break
    else:
        print(f"Movie '{user_movie_name}' not found. Please enter a valid movie name.")
    user_movie_name = input("Enter a movie name: ")

if user_movie_name in movies_df['title'].values:
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(movies_df['genres'].fillna(''))

    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

    movie_index = movies_df[movies_df['title'] == user_movie_name].index[0]

    similar_movies = list(enumerate(cosine_sim[movie_index]))

    similar_movies.sort(key=lambda x: x[1], reverse=True)

    user_director_name = input("\nEnter the director's name: ")

    if not user_director_name.strip():
        print("\nMovie details for similar movies with similarity greater than 0%:")
        for idx, score in similar_movies[1:]:
            if score > 0:
                movie_title = movies_df.iloc[idx]['title']
                movie_overview = movies_df.iloc[idx]['overview']
                movie_homepage = movies_df.iloc[idx]['homepage']
                movie_cast = movies_df.iloc[idx]['cast']

                print(f"\n{movie_title} ({score:.2%}): {movie_overview}")
                print(f"Homepage: {movie_homepage}")
                print(f"Cast: {movie_cast}")

    else:
        director_movies = movies_df[movies_df['director'] == user_director_name]['title'].tolist()
        if director_movies:
            print("\nMovies directed by", user_director_name)
            for movie in director_movies:
                movie_row = movies_df[movies_df['title'] == movie]
                overview = movie_row['overview'].values[0] if not movie_row['overview'].empty else "No overview available"
                homepage = movie_row['homepage'].values[0] if not movie_row['homepage'].empty else "No homepage available"
                cast = movie_row['cast'].values[0] if not movie_row['cast'].empty else "No cast information available"
                print(f"{movie}: {overview}")
                print(f"Homepage: {homepage}")
                print(f"Cast: {cast}")

    if not user_movie_name.strip() and not user_director_name.strip():
        print("\nYou didn't pick anything.")


Enter a movie name:  Iron Man

Enter the director's name:  



Movie details for similar movies with similarity greater than 0%:

Avengers: Age of Ultron (100.00%): When Tony Stark tries to jumpstart a dormant peacekeeping program, things go awry and Earth’s Mightiest Heroes are put to the ultimate test as the fate of the planet hangs in the balance. As the villainous Ultron emerges, it is up to The Avengers to stop him from enacting his terrible plans, and soon uneasy alliances and unexpected action pave the way for an epic and unique global adventure.
Homepage: http://marvel.com/movies/movie/193/avengers_age_of_ultron
Cast: Robert Downey Jr. Chris Hemsworth Mark Ruffalo Chris Evans Scarlett Johansson

The Avengers (100.00%): When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!
Homepage: http://marvel.com